In [1]:
import os
import pandas as pd
import numpy as np
from typing import Dict

from tdc.utils.retrieve import retrieve_label_name_list
from scripts.tdc_constants import (
    DATASET_TO_TYPE,
    DATASET_TO_LABEL_NAMES
)

from sklearn.model_selection import StratifiedKFold, train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def save_distribution_plot(train_df, val_df, test_df, target_col, task_name, fold_idx, fold_dir):
    """
    Visualizes and saves the label distribution for Train, Val, and Test sets.
    """
    plt.figure(figsize=(7, 5))
    
    # Represent regression data with a continuous distribution (KDE) 
    # and classification data with discrete bars
    sns.histplot(train_df[target_col], kde=True, label="Train", color='blue', alpha=0.3)
    sns.histplot(test_df[target_col], kde=True, label="Test", color='red', alpha=0.3)
    sns.histplot(val_df[target_col], kde=True, label="Val", color='green', alpha=0.3)

    plt.title(f"{task_name} Fold-{fold_idx} Distribution")
    plt.xlabel(target_col)
    plt.ylabel("Frequency")
    plt.legend()
    plt.tight_layout()
    
    save_path = os.path.join(fold_dir, f"{task_name}_distribution.png")
    plt.savefig(save_path)
    plt.close()

In [3]:
def create_stratified_bins(df, target_col, task_type, n_splits=5):
    """
    Divides regression data into quantile-based bins for stratification.
    Uses original labels for classification tasks.
    """
    df = df.copy()
    if task_type == "regression":
        # Create bins based on quantiles for regression to enable StratifiedKFold
        df['strat_label'] = pd.qcut(
            df[target_col], q=n_splits, labels=False, duplicates="drop"
        )
    else:
        # For classification, use the existing binary/categorical labels
        df['strat_label'] = df[target_col]
    return df

In [4]:
def run_label_cv(df, dataset_name, label_name, output_root, n_splits=5, val_ratio=0.1, random_state=2026):
    """
    Performs 5-Fold CV. Automatically detects the target column if it doesn't match label_name.
    """
    # --- Dynamic Target Column Detection ---
    if label_name in df.columns:
        target_col = label_name
    elif 'hERG_inhib' in df.columns: # Specific fix for herg_central
        target_col = 'hERG_inhib'
    else:
        # Fallback: Assume the target is the column that isn't 'smiles'
        cols = [c for c in df.columns if c.lower() != 'smiles']
        target_col = cols[0]
        print(f"  [Note] Mapping label '{label_name}' to CSV column '{target_col}'")

    task_type = DATASET_TO_TYPE.get(label_name, "classification")
    label_root = os.path.join(output_root, dataset_name, label_name)
    os.makedirs(label_root, exist_ok=True)

    # --- Handle Duplicate SMILES ---
    if df['smiles'].duplicated().any():
        df = df.groupby('smiles', as_index=False)[target_col].mean()

    if task_type == "classification":
        df[target_col] = df[target_col].round().astype(int)

    # --- Prepare Stratification ---
    # Pass target_col instead of label_name
    df_prepared = create_stratified_bins(df, target_col, task_type, n_splits)
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    fold_summaries = []

    for fold_idx, (train_idx, test_idx) in enumerate(skf.split(df_prepared, df_prepared['strat_label']), start=1):
        fold_dir = os.path.join(label_root, f"fold{fold_idx}")
        os.makedirs(fold_dir, exist_ok=True)

        train_val_df = df_prepared.iloc[train_idx]
        test_df = df_prepared.iloc[test_idx]

        train_df, val_df = train_test_split(
            train_val_df, test_size=val_ratio, 
            stratify=train_val_df['strat_label'], random_state=random_state
        )

        # Save Plot
        save_distribution_plot(train_df, val_df, test_df, target_col, label_name, fold_idx, fold_dir)

        # Drop temporary column and Save CSVs
        for name, d in [("train", train_df), ("val", val_df), ("test", test_df)]:
            d_save = d.drop(columns=['strat_label'], errors='ignore').copy()
            # Ensure the saved CSV has the label_name as header for consistency
            d_save = d_save.rename(columns={target_col: label_name})
            d_save.to_csv(os.path.join(fold_dir, f"{name}.csv"), index=False)

        summary = {
            "fold": fold_idx, 
            "train_size": len(train_df), 
            "val_size": len(val_df), 
            "test_size": len(test_df),
            "mean": round(train_df[target_col].mean(), 4)
        }
        fold_summaries.append(summary)

    pd.DataFrame(fold_summaries).to_csv(os.path.join(label_root, "fold_summary.csv"), index=False)

In [5]:
def get_dataset_labels(dataset_name):
    if dataset_name in DATASET_TO_LABEL_NAMES:
        return DATASET_TO_LABEL_NAMES[dataset_name]
    return [dataset_name]

In [6]:
def start_admet_cv_process(dataset_names, raw_root, split_root):
    for dataset in dataset_names:
        labels = get_dataset_labels(dataset)
        for label in labels:
            file_path = os.path.join(raw_root, dataset, f"{label}.csv")
            if not os.path.exists(file_path):
                continue
            
            print(f">>> Processing: {dataset} | Label: {label}")
            df = pd.read_csv(file_path)
            run_label_cv(df, dataset, label, split_root)

In [7]:
def process_data_directory(raw_root, split_root, filter_list=None):
    """
    Scans for dataset folders and processes CSV files.
    Includes a hardcoded path exception for the 'herg_central' dataset.
    """
    if not os.path.exists(raw_root):
        print(f"Error: Directory {raw_root} does not exist.")
        return

    # List all items in the raw_root
    all_items = os.listdir(raw_root)

    for item in all_items:
        # Construct the full path to check if it's a directory
        item_path = os.path.join(raw_root, item)
        
        if not os.path.isdir(item_path):
            continue
            
        # Apply filter_list if provided
        if filter_list and item not in filter_list:
            continue

        # --- Hardcoded Exception for herg_central ---
        if item == "herg_central":
            dataset = "herg_central"
            label_name = "herg_central_hERG_inhib"
            file_path = os.path.join(raw_root, dataset, f"{label_name}.csv")
            
            if os.path.exists(file_path):
                print(f">>> Processing (Hardcoded): {dataset} / {label_name}")
                df = pd.read_csv(file_path)
                run_label_cv(df, dataset, label_name, split_root)
            else:
                print(f"  [Error] Hardcoded file not found: {file_path}")
            continue # Skip the rest of the loop for this item

        # --- Standard Logic for Other Folders ---
        csv_files = [f for f in os.listdir(item_path) if f.endswith('.csv')]

        for csv_file in csv_files:
            label_name = csv_file.replace('.csv', '')
            file_path = os.path.join(item_path, csv_file)
            
            print(f">>> Processing: {item} / {label_name}")
            df = pd.read_csv(file_path)
            
            # Execute the Cross-Validation pipeline
            run_label_cv(df, item, label_name, split_root)

In [8]:
RAW_DATA_PATH = "./data/tdc_admet_all"
OUTPUT_SPLIT_PATH = "./data/processed/"

In [9]:
# Optional: Define specific datasets to process, or set to None to process ALL
# MY_DATASETS = [
#     'Caco2_Wang', 'Pgp_Broccatelli', 'VDss_Lombardo', 'BBB_Martins',
#     'Half_life_Obach', 'LD50_Zhu', 'hERG', 'AMES', 'ClinTox', 'Tox21'
# ]

# process_data_directory(
#     raw_root=RAW_DATA_PATH,
#     split_root=OUTPUT_SPLIT_PATH,
#     filter_list=MY_DATASETS # Use None to process everything in the folder
# )

In [10]:
process_data_directory(
    raw_root=RAW_DATA_PATH,
    split_root=OUTPUT_SPLIT_PATH,
    filter_list=None # Use None to process everything in the folder
)

>>> Processing: Lipophilicity_AstraZeneca / Lipophilicity_AstraZeneca
>>> Processing: Solubility_AqSolDB / Solubility_AqSolDB
>>> Processing: Clearance_Microsome_AZ / Clearance_Microsome_AZ
>>> Processing: CYP2D6_Substrate_CarbonMangels / CYP2D6_Substrate_CarbonMangels
>>> Processing: HIA_Hou / HIA_Hou
>>> Processing: CYP2C9_Substrate_CarbonMangels / CYP2C9_Substrate_CarbonMangels
>>> Processing: PPBR_AZ / PPBR_AZ
>>> Processing: CYP3A4_Veith / CYP3A4_Veith
>>> Processing (Hardcoded): herg_central / herg_central_hERG_inhib
>>> Processing: Caco2_Wang / Caco2_Wang
>>> Processing: DILI / DILI
>>> Processing: hERG / hERG
>>> Processing: Carcinogens_Lagunin / Carcinogens_Lagunin
>>> Processing: CYP2D6_Veith / CYP2D6_Veith
>>> Processing: ClinTox / ClinTox
>>> Processing: CYP3A4_Substrate_CarbonMangels / CYP3A4_Substrate_CarbonMangels
>>> Processing: Skin_Reaction / Skin_Reaction
>>> Processing: hERG_Karim / hERG_Karim
>>> Processing: Half_Life_Obach / Half_Life_Obach
>>> Processing: Hydrati

In [11]:
import os
from pathlib import Path

def check_folder_consistency(dir1_path, dir2_path):
    dir1 = Path(dir1_path)
    dir2 = Path(dir2_path)

    # Get sets of filenames (ignoring subdirectories if any)
    files1 = {f.name for f in dir1.iterdir() if f.is_file()}
    files2 = {f.name for f in dir2.iterdir() if f.is_file()}

    print(f"--- Folder Consistency Report ---")
    print(f"Folder 1: {dir1} ({len(files1)} files)")
    print(f"Folder 2: {dir2} ({len(files2)} files)")
    print("-" * 33)

    if files1 == files2:
        print("✅ Success: Both folders have identical filenames and counts.")
    else:
        print("❌ Mismatch Detected!")
        
        # Files in dir1 but not in dir2
        only_in_1 = files1 - files2
        if only_in_1:
            print(f"- Missing in Folder 2 ({len(only_in_1)}): {sorted(list(only_in_1))}")

        # Files in dir2 but not in dir1
        only_in_2 = files2 - files1
        if only_in_2:
            print(f"- Missing in Folder 1 ({len(only_in_2)}): {sorted(list(only_in_2))}")

RAW_DIR = "/home/rlawlsgurjh/hdd/work/QSAR/admet_ai/data/tdc_admet_all"
SPLIT_DIR = "/home/rlawlsgurjh/hdd/work/QSAR/admet_ai/data/processed"

check_folder_consistency(RAW_DIR, SPLIT_DIR)

--- Folder Consistency Report ---
Folder 1: /home/rlawlsgurjh/hdd/work/QSAR/admet_ai/data/tdc_admet_all (0 files)
Folder 2: /home/rlawlsgurjh/hdd/work/QSAR/admet_ai/data/processed (0 files)
---------------------------------
✅ Success: Both folders have identical filenames and counts.
